**SOBRE O CONJUNTO**  
Este conjunto de dados apresenta uma lista de reclamações de consumidores apresentadas contra seguradoras licenciadas em Connecticut (Estado dos EUA). O mesmo inclui a empresa, linha de negócios, natureza da reclamação, resultado ou resolução e recuperação.  

**PROJETO**  
Este projeto sob a perspectiva de estatística descritiva e probabilística, demonstrar de forma prática e visual, dados acerca do dataset [Insurance Company Complaints](https://www.kaggle.com/datasets/adelanseur/insurance-company-complaints). Ao final, serão respondidas algumas perguntas de negócio, acerca das questões levantadas.

<u>VARIÁVEIS</u>:  
**Company**: nome da seguradora envolvida na reclamação.  
**Opened**: data em que a reclamação foi aberta ou recebida.  
**Closed**: data em que a reclamação foi resolvida e oficialmente encerrada.  
**Coverage**: tipo de cobertura de seguro relacionada à reclamação (por exemplo, seguro de automóvel, seguro de saúde, etc.).  
**SubCoverage**: maiores detalhes ou subcategoria da cobertura do seguro.  
**Reason**: principal motivo ou causa da reclamação.  
**SubReason**: maiores detalhes ou subcategoria do motivo da reclamação.     
**Recovery**: quantia de dinheiro recuperada (se aplicável), como resultado da resolução da reclamação.  
**Status** : status atual da reclamação (por exemplo, aberta, fechada, em andamento, etc.).  

# PREPARAÇÃO, ORGANIZAÇÃO E ESTRUTURAÇÃO DOS DADOS

## Extração e Limpeza de Dados

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/programacao/datasets/reclamacoes_seguradoras/iccrsr.csv")
df.head()

,Company,File No.,Opened,Closed,Coverage,SubCoverage,Reason,SubReason,Disposition,Conclusion,Recovery,Status
0,"Anthem Health Plans, Inc",7045593,05/31/2022,06/02/2022,Group,Health Only,Claim Handling,Medically Necessary,Company Position Substantiated,Company Position Upheld,0.00,Closed
1,"Anthem Health Plans, Inc",7043381,02/28/2022,06/02/2022,Group,Health Only,Claim Handling,Provider Contract Issue,Claim Settled,Satisfied,6467.30,Closed
2,"Anthem Health Plans, Inc",7044860,05/03/2022,06/02/2022,A & H,Health Only,Claim Handling,Denial,Claim Settled,Claim Paid,147.58,Closed
3,"Anthem Health Plans, Inc",7043381,02/28/2022,06/02/2022,Group,A & H,Claim Handling,Provider Contract Issue,Claim Settled,Satisfied,6467.30,Closed
4,"Anthem Health Plans, Inc",7052007,02/23/2023,03/17/2023,A & H,A & H,Marketing & Sales,Duplicate Coverage,Compromised Settlement/Resolution,Premium Refund,2179.32,Closed


Primeiramente, será verificado a quantidade de variáveis e registros contidos no dataset.

In [3]:
df.shape

(38267, 12)

Também é importante verificar o tipo dos dados.

In [4]:
df.dtypes

Company         object
File No.         int64
Opened          object
Closed          object
Coverage        object
SubCoverage     object
Reason          object
SubReason       object
Disposition     object
Conclusion      object
Recovery       float64
Status          object
dtype: object

As variáveis Opened e Closed devem ser modificadas para o tipo *datetime*.

In [5]:
df["Opened"] = pd.to_datetime(df["Opened"])
df["Closed"] = pd.to_datetime(df["Closed"])

<u>Remoção de variáveis e registros</u>  
Para a remoção de dados, primeiramente, será verificado valores nulos.

In [6]:
df.isnull().sum()

Company            0
File No.           0
Opened             0
Closed           963
Coverage        2440
SubCoverage     8960
Reason          2617
SubReason       2617
Disposition    15288
Conclusion     17094
Recovery           0
Status             0
dtype: int64

Há variáveis com valores nulos, porém, apenas algumas são de suma importância para o projeto e, estas devem ser removidas.
Como não será feito nenhum processo de machine learning, não é interessante manter valores nulos acerca da variável Opened.

In [7]:
df.dropna(subset=["Closed"], inplace=True)

In [8]:
df.shape

(37304, 12)

In [9]:
df.reset_index(drop=True, inplace=True)

# ESTATÍSTICA DESCRITIVA

## Distribuição de Frequências
É interessante começar verificando a exclusividade dos dados.

In [10]:
df.nunique()

Company          755
File No.       20029
Opened          1593
Closed          1377
Coverage          53
SubCoverage      101
Reason             4
SubReason        177
Disposition       13
Conclusion        52
Recovery        2742
Status             9
dtype: int64

Olhando para a exclusividade em cada variável, é possível ter um ponto de partida para a descrição dos dados.  
### Seguradora x Reclamações
O dataset gira em torno de reclamações acerca de seguradoras, portanto, a descrição dos dados iniciará sob as seguradoras com maior número de reclamações.

In [ ]:
cols_segs = {"index": "Seguradora", "Company": "Quantidade"}

In [11]:
df_segs = df.Company.value_counts()[:10].reset_index().rename(columns=cols_segs)

df_segs

,Seguradora,Quantidade
0,"Anthem Health Plans, Inc",6519
1,"ConnectiCare Benefits, Inc.",1963
2,UnitedHealthcare Insurance Company,1503
3,"ConnectiCare Insurance Company, Inc",1186
4,Cigna Health and Life Insurance Company,1151
5,Allstate Fire and Casualty Insurance Company,892
6,"Oxford Health Plans (CT), Inc",855
7,State Farm Mutual Automobile Insurance Company,775
8,Progressive Direct Insurance Company,768
9,ConnectiCare Inc,626


A informação obtida desta operação pode ser visualizada para um melhor entendimento. Um gráfico aceitável para tal, que demonstra de forma objetiva o percentual de ocorrências das reclamações.

In [12]:
import plotly.graph_objects as go
import plotly.express as px

In [13]:
tema = px.colors.qualitative.Light24

In [14]:
labels = df_segs.Seguradora.values
values = df_segs.Quantidade.values

In [15]:
graf_segs = go.Figure(go.Pie(labels=labels, values=values, hole=0.25,
                             texttemplate="%{label}<br>%{percent}",
                             marker=dict(colors=tema)))

graf_segs.update_layout(font_size=12, showlegend=False,
                        title=dict(text="SEGURADORA x QUANTIDADE DE RECLAMAÇÃO",
                                   x=0.5, font_size=22),
                        annotations=[dict(text=f"<b>TOTAL<br>{sum(values)}</b>",
                                          x=0.5, y=0.5, font_size=14,
                                          showarrow=False)])

Com isso, se observa uma grande discrepância acerca das reclamções. A seguradora *Anthem Health Plans, Inc* possui um percentual elevado de reclamações, se comparado as demais, alcançando cerca de 40,1% dos dados, isso equivale a 6519 dos 16238 registros.  
### Motivo ou causa de reclamação
Outro fator importante a ser entendido é o principal motivo ou causa da reclamação. Esta informação pode ser obtida a partir da variável *Reason*.

In [ ]:
cols_recls = {"index": "Reclamação", "Reason": "Quantidade"}

In [16]:
df_recls = df.Reason.value_counts().reset_index().rename(columns=cols_recls)

df_recls

,Reclamação,Quantidade
0,Claim Handling,26124
1,PolicyHolder Service,3915
2,Underwriting,3604
3,Marketing & Sales,1225


Os motivos ou causas de reclamação são: Claim Handling (Manuseio de Reclamações), PolicyHolder Service (Atendimento ao Titular da Apólice), Underwriting (Subscrição) e Marketing & Sales (Marketing & Vendas). Dentre elas, a predominante é *Claim Handling*. A visualização desstas frequências pode ser feita, de modo a facilitar o entendimento dos dados, a partir de um gráfico de barras.

In [17]:
graf_reason = px.histogram(df_recls, x="Quantidade", y="Reclamação", width=750,
                           height=400, orientation='h', text_auto=True,
                           color="Reclamação", color_discrete_sequence=tema)

graf_reason.update_layout(font_size=12, showlegend=False,
                          title=dict(text="PRINCIPAIS MOTIVOS OU CAUSAS DE RECLAMAÇÃO",
                                     x=0.5, font_size=22),
                          plot_bgcolor="rgb(250, 250, 250)")

graf_reason.update_xaxes(showline=True, linewidth=1, linecolor="lightgrey",
                         title="Quantidade")

graf_reason.update_yaxes(showline=True, linewidth=1, linecolor="lightgrey",
                         title=None, categoryorder="total descending")

Observando a distribuição, fica evidente a discrepância acerca do principal motivo ou causa de reclamação, em relação aos demais. Além disso, é possível se ver as principais sub-reclamações, para se ter um entendimento maior sobre a variável *Reason*, encontrada na variável *SubReason*.  
<u>**Manuseio de Reclamações (Claim Handling)**</u>  
Área responsável por lidar com os processos e procedimentos quando um segurado precisa fazer uma reclamação devido a um evento coberto pela apólice.

In [18]:
df[df.Reason == "Claim Handling"].SubReason.value_counts()[:5]

Claim Denial                       4501
Claim Delay                        3837
Unsatisfactory Settlement/Offer    3250
Claim Procedure                    1701
Medical Necessity Denial           1307
Name: SubReason, dtype: int64

<u>**Atendimento ao Titular da Apólice (PolicyHolder Service)**</u>  
Engloba todos os aspectos de comunicação e assistência fornecidos aos clientes que possuem uma apólice de seguro.

In [19]:
df[df.Reason == "PolicyHolder Service"].SubReason.value_counts()[:5]

Premium Notice/Billing              1012
Delays/No Response                   712
Unsatisfactory Refund of Premium     432
Premium/Notice                       312
Premium Refund Delay                 159
Name: SubReason, dtype: int64

<u>**Subscrição (Underwriting)**</u>  
Processo pelo qual a seguradora avalia o risco de cobrir um cliente em potencial.

In [20]:
df[df.Reason == "Underwriting"].SubReason.value_counts()[:5]

Premium & Rating         1544
Cancellation              669
Nonrenewal                496
Premium/Rate Increase     288
No Subreason              153
Name: SubReason, dtype: int64

<u>**Marketing & Vendas (Marketing & Sales)**</u>  
Responsável por promover os produtos e serviços da seguradora, bem como atrair novos clientes.

In [21]:
df[df.Reason == "Marketing & Sales"].SubReason.value_counts()[:5]

High Pressure Tactics          241
Misrepresentation              185
Misappropriation of Premium    163
No Coverage/Premium Paid       144
Misleading Advertising          71
Name: SubReason, dtype: int64

### Cobertura de seguro x Motivo de reclamação
Através da variável *Coverage*, é determinado o tipo de cobertura de seguro. Com isso, será feita uma análise sobre a mesma, em relação ao motivo ou causa de reclamação.

In [22]:
cob_seg = df.Coverage.value_counts()

i_cob_seg = cob_seg[cob_seg > 1000].index

In [23]:
df_cob_seg = df[df.Coverage.isin(i_cob_seg)]

In [24]:
graf_cob_seg = px.histogram(df_cob_seg, x="Coverage", width=600, height=700,
                            text_auto=True, color="Reason",
                            color_discrete_sequence=tema)

graf_cob_seg.update_layout(font_size=12, showlegend=False,
                           title=dict(text="COBERTURA DE SEGURO x MOTIVO DE RECLAMAÇÃO",
                                      x=0.5, font_size=22),
                           plot_bgcolor="rgb(250, 250, 250)")

graf_cob_seg.update_xaxes(showline=True, linewidth=1, linecolor="lightgrey",
                          title=None, categoryorder="total descending")

graf_cob_seg.update_yaxes(showline=True, linewidth=1, linecolor="lightgrey",
                          title="Quantidade")

Com isso, se constata que os tipos de coberturas de seguros mais solicitados, tem como principal motivo ou causa de reclamação a "Claim Handling", com excessão de *Individual Life*, que tem PolicyHolder Service como principal motivo ou causa de reclamação.
### Reclamações por dia
Agora, se discorrerá sobre o número de abertura de reclamações (Opened) por dia. Com isso, será possível entender se houve picos, altas, baixas, a respeito das reclamações.

In [25]:
recs_dia = df.sort_values(by="Opened").Opened.value_counts(sort=False)

In [26]:
graf_recs_dia = go.Scatter(x=recs_dia.index, y=recs_dia.values, mode="lines",
                           line=dict(color=tema[0]))

layout_recs = go.Layout(title=dict(text="ABERTURA DE RECLAMAÇÕES POR DIA",
                                   x=0.5, font_size=22),
                        yaxis=dict(title="Reclamação (ões)"), height=600)

fig_recs_dia = go.Figure(data=graf_recs_dia, layout=layout_recs)

fig_recs_dia.update_layout(xaxis_range=["2020", "2023-06"],
                           plot_bgcolor="#F9F9F9")

fig_recs_dia.update_yaxes(showline=True, linewidth=1, linecolor="lightgrey")

fig_recs_dia.update_xaxes(showline=True, linewidth=1, linecolor="lightgrey",
                          rangeslider_visible=True)

Olhando para o gráfico, é possível ver um aumento exponencial a partir de novembro de 2021. Além disso, se nota que, até o dia 31 de outubro de 2017, há poucos registros de reclamações.

# ESTATÍSTICA PROBABILÍSTICA
Aqui, será visto as chances de determinado evento acontecer.

## Perguntas de Negócio
Sob a perspectiva de perguntas de negócio, serão resolvidas algumas questões a partir do conjunto de dados.

**1) Qual a probabilidade de um cliente com o tipo de cobertura de seguro *Individual* ter seu problema resolvido em menos 20 dias?**

In [27]:
prob_a = lambda a, e: round((a / e) * 100, 2)
e1 = df[df.Coverage == "Individual"]

In [29]:
a1 = len([i.days for i in e1.Closed - e1.Opened if i.days < 20])

In [30]:
print(f"R = Há {prob_a(a1, len(e1))}% de chance do evento ocorrer.")

R = Há 36.56% de chance do evento ocorrer.


**2) Clientes com cobertura do tipo group tem quantos por cento de chance de não ter problemas de *Claim Handling*?**

In [31]:
prob_nao = lambda a, e: round((1 - (a / e)) * 100, 2)
e2 = df[df.Coverage == "Group"]

In [33]:
a2 = len(e2[e2.Reason == "Claim Handling"])

In [34]:
print(f"R = Há {prob_nao(a2, len(e2))}% de chance do evento não ocorrer.")

R = Há 10.62% de chance do evento não ocorrer.


**3) Houve quantas aberturas de reclamação por dia em média?**

In [35]:
md_dia = int(df.Opened.value_counts().mean())

In [36]:
print(f"R = Houve em média {md_dia} aberturas de reclamação por dia.")

R = Houve em média 23 aberturas de reclamação por dia.


**4) Um cliente que possui a cobertura *Homeowners* tem quantos por cento de chance de ter seu caso reaberto (Reopened).**

In [37]:
e4 = df[df.Coverage == "Homeowners"]

In [38]:
a4 = len(e4[e4.Status == "Reopened"])

In [39]:
print(f"R = Há {prob_a(a4, len(e4))}% de chance do evento ocorrer.")

R = Há 0.03% de chance do evento ocorrer.


**5) Os casos foram resolvidos em média em quantos dias**

In [40]:
dias = [i.days for i in df.Closed - df.Opened]

In [41]:
md_casos = int(sum(dias) / len(dias))

In [42]:
print(f"R = Demorou em média {md_casos} dias para a resolução dos problemas.")

R = Demorou em média 41 dias para a resolução dos problemas.


**6) Ao optar pelo plano *Individual* da seguradora *Anthem Health Plans, Inc*, qual a chance de se ter um problema de *PolicyHolder Service*?**

In [43]:
e6 = df.query("Company == 'Anthem Health Plans, Inc' and Coverage == 'Individual'")

a6 = len(e6[e6.Reason == "PolicyHolder Service"])

In [44]:
print(f"R = Há {prob_a(a6, len(e6))}% de chance do evento ocorrer.")

R = Há 13.79% de chance do evento ocorrer.


**7) Qual a probabilidade de se ter problema de *Claim Handling* ou *PolicyHolder Service* ao se contratar o serviço de uma seguradora?**

In [51]:
prob_uniao = lambda pA, pB: pA + pB
e8 = len(df.Reason)

In [49]:
pA = prob_a(len(df.query("Reason == 'Claim Handling'")), e8)
pB = prob_a(len(df[df.Reason == "PolicyHolder Service"]), e8)

In [54]:
print(f"R = Há {prob_uniao(pA, pB)}% de chance do evento ocorrer.")

R = Há 80.52% de chance do evento ocorrer.


**8) Em geral, qual foi a média de recuperação de valores?**

In [45]:
md_rec = prob_a(len(df[df.Recovery > 0]), len(df))

In [46]:
print(f"R = Houve em média {md_rec}% de recuperação de valores.")

R = Houve em média 20.76% de recuperação de valores.
